In [1]:
# George Miloshevich 2022
# The point of this notebook is to show that the new loss and the old loss give the same result
# This routine is written for two parameters: input folder for VAE weights and the given epoch. It shows us how good the reconstruction of the VAE works
import os, sys
import shutil
from pathlib import Path
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'  # https://stackoverflow.com/questions/65907365/tensorflow-not-creating-xla-devices-tf-xla-enable-xla-devices-not-set
import logging
from colorama import Fore # support colored output in terminal
from colorama import Style
if __name__ == '__main__':
    logger = logging.getLogger()
    logger.handlers = [logging.StreamHandler(sys.stdout)]
else:
    logger = logging.getLogger(__name__)
logger.level = logging.INFO

fold_folder = Path('./xforanalogs/tests/testL2linear/fold_0')  # The name of the folder where the weights have been stored
checkpoint = 15       # The checkpoint at which the weights have been stored

import importlib.util
def module_from_file(module_name, file_path): #The code that imports the file which originated the training with all the instructions
            spec = importlib.util.spec_from_file_location(module_name, file_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            return module
logger.info(f"{Fore.BLUE}") #  indicates we are inside the routine       
logger.info(f"{fold_folder = }")
logger.info(f"loading module from  {fold_folder.parent}/Funs.py")
from importlib import import_module
#foo = import_module(fold_folder+'/Funs.py', package=None)
foo = module_from_file("foo", f'{fold_folder.parent}/Funs.py')
ef = foo.ef # Inherit ERA_Fields_New from the file we are calling
ln = foo.ln
ut = foo.ut

run_vae_kwargs = ut.json2dict(f"{fold_folder.parent}/config.json")

logger.info("==Importing tensorflow packages===")
import random as rd  
from scipy.stats import norm
import numpy as np


if len(sys.argv) > 3:
    rd.seed(a=int(sys.argv[3]))
else:
    rd.seed(a=None) # None = system time

tff = foo.tff # tensorflow routines 
ut = foo.ut # utilities
logger.info("==Checking GPU==")
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

logger.info("==Checking CUDA==")
tf.test.is_built_with_cuda()

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.mpl.geoaxes

import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeat
data_proj = ccrs.PlateCarree()

#from tensorflow.keras.preprocessing.image import ImageDataGenerator
sys.path.insert(1, '../ERA')
import cartopy_plots as cplt


logger.info("==Reading data==")

year_permutation = np.load(f'{fold_folder.parent}/year_permutation.npy')
i = int(np.load(f'{fold_folder}/fold_num.npy'))

time_start = ut.extract_nested(run_vae_kwargs, 'time_start')
time_end = ut.extract_nested(run_vae_kwargs, 'time_end')
T = ut.extract_nested(run_vae_kwargs, 'T')
if (ut.keys_exists(run_vae_kwargs, 'label_period_start') and ut.keys_exists(run_vae_kwargs, 'label_period_end')):
    label_period_start = ut.extract_nested(run_vae_kwargs, 'label_period_start')
    label_period_end = ut.extract_nested(run_vae_kwargs, 'label_period_end')
    if (label_period_start is not None) and (label_period_end is not None):
        summer_days = label_period_end - label_period_start - T + 1
    elif (label_period_start is None) and (label_period_end is not None):
        summer_days = label_period_end - time_start - T + 1
    elif (label_period_start is not None) and (label_period_end is None):
        summer_days = time_end - label_period_start - T + 1
    else:
        summer_days = time_end - time_start - T + 1
else:
    summer_days = time_end - time_start - T + 1
if ut.keys_exists(run_vae_kwargs, 'normalization_mode'):
    normalization_mode = ut.extract_nested(run_vae_kwargs, 'normalization_mode')
else:
    normalization_mode = None

if ut.keys_exists(run_vae_kwargs, 'keep_dims'):
    keep_dims = ut.extract_nested(run_vae_kwargs, 'keep_dims')
else:
    keep_dims = None

logger.info(f"{Style.RESET_ALL}")
run_vae_kwargs = ut.set_values_recursive(run_vae_kwargs, {'myinput' : 'N'})
if not os.path.exists(ut.extract_nested(run_vae_kwargs, 'mylocal')): # we are assuming that training was not run on R740server5
    run_vae_kwargs = ut.set_values_recursive(run_vae_kwargs, {'mylocal' : '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe/PLASIM/'})
logger.info(f"{run_vae_kwargs = }")

history, N_EPOCHS, INITIAL_EPOCH, checkpoint_path, LAT, LON, vae, X_va, Y_va, X_tr, Y_tr, _ = foo.run_vae(fold_folder, **run_vae_kwargs)




fold_folder = PosixPath('xforanalogs/tests/testL2linear/fold_0')
loading module from  xforanalogs/tests/testL2linear/Funs.py
Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


2022-11-16 17:31:53.400099: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


path_to_ERA = '/ClimateDynamics/MediumSpace/ClimateLearningFR/gmiloshe'/ERA/
Trying to import basemap
In this environment you cannot import Basemap
Trying to import cartopy
Trying to import basemap
In this environment you cannot import Basemap
Trying to import cartopy
Successfully imported cartopy
Successfully imported cartopy
==Checking GPU==
From xforanalogs/tests/testL2linear/Funs.py:45: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
==Checking CUDA==
==Importing tensorflow packages===
==Checking GPU==
==Checking CUDA==
==Reading data==

run_vae_kwargs = {'myinput': 'N', 'XY_run_vae_keywargs': None, 'log_level': 20, 'k_fold_cross_val_kwargs': {'keep_dims': [1], 'nfolds': 10, 'val_folds': 1, 'range_nfolds': None, 'u': 1, 'normalization_mode': 'global_logit', 'classification': True, 'evaluate_epoch': 'last', 'repeat_nan': 5, 'create_or_

2022-11-16 17:32:04.501971: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 17:32:04.600009: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300000000 Hz
2022-11-16 17:32:04.659120: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55edf06be110 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-11-16 17:32:04.659173: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-11-16 17:32:04.663691: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-16 17:32:04.968800: E tensorflow/stream_execu

    prepare_data_and_mask:
        load_data:
            Opening field tas
            monotonize_years:
            monotonize_years: completed in 3.9 s
            select_years:
            select_years: completed in 0.7 s
            select_lonlat:
            select_lonlat: completed in 2.8 s
            Opening field zg
            monotonize_years:
            monotonize_years: completed in 4.2 s
            select_years:
            select_years: completed in 0.7 s
            select_lonlat:
            select_lonlat: completed in 2.9 s
            Opening field mrso
            monotonize_years:
            monotonize_years: completed in 4.0 s
            select_years:
            select_years: completed in 0.7 s
            select_lonlat:
            select_lonlat: completed in 2.7 s
        load_data: completed in 1 min 36.3 s
        prepare_XY:
            return_threshold = True            
            make_XY:
                make_X:
                make_X: completed in 

In [2]:
X_va.shape, np.max(X_va), np.min(X_va)

((52500, 24, 48, 1), 7.5999355, -6.233911)

In [3]:
data = X_va[0:2,...]
z_mean, z_log_var, z, zz = vae.call_encoder_classifier(data)
reconstruction = vae.decoder(z)
reconstruction.shape

TensorShape([2, 24, 48, 1])

In [4]:
data = X_va[0:2,...]
factor = vae.k1*vae.encoder_input_shape[1]*vae.encoder_input_shape[2]
vae.coef_out*factor*tf.reduce_mean([tf.reduce_mean(vae.bce(data[...,i][..., np.newaxis], reconstruction[...,i][..., np.newaxis])) for i in range(reconstruction.shape[3])] ) 

<tf.Tensor: shape=(), dtype=float32, numpy=-158404.47>

In [5]:
vae.k1*tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstruction, from_logits=vae.from_logits), axis=(1, 2) # -Y_n Log( P_n) - (1 - Y_n) Log( 1 - P_n) is the expression for binary entropy
                )
            )

<tf.Tensor: shape=(), dtype=float32, numpy=-158404.47>

In [7]:
%%time
data = X_va
z_mean, z_log_var, z, zz = vae.call_encoder_classifier(data)
reconstruction = vae.decoder(z)
reconstruction.shape
factor = vae.k1*vae.encoder_input_shape[1]*vae.encoder_input_shape[2]
vae.coef_out*factor*tf.reduce_mean([tf.reduce_mean(vae.rec_loss_form(data[...,i][..., np.newaxis], reconstruction[...,i][..., np.newaxis])) for i in range(reconstruction.shape[3])] ) 

CPU times: user 1min 12s, sys: 1min 45s, total: 2min 57s
Wall time: 3.67 s


<tf.Tensor: shape=(), dtype=float32, numpy=1476.3168>

In [21]:
load_data_kwargs = ut.extract_nested(run_vae_kwargs, 'load_data_kwargs')
prepare_XY_kwargs = ut.extract_nested(run_vae_kwargs, 'prepare_XY_kwargs')
(_, _, year_permutation, _, _, _, threshold), mask  = ln.prepare_data_and_mask(load_data_kwargs=load_data_kwargs, prepare_XY_kwargs=prepare_XY_kwargs)

prepare_data_and_mask:
    load_data:
        Opening field tas
        monotonize_years:
        monotonize_years: completed in 4.4 s
        select_years:
        select_years: completed in 0.8 s
        select_lonlat:
        select_lonlat: completed in 2.9 s
        Opening field zg
        monotonize_years:
        monotonize_years: completed in 4.0 s
        select_years:
        select_years: completed in 0.7 s
        select_lonlat:
        select_lonlat: completed in 2.9 s
        Opening field mrso
        monotonize_years:
        monotonize_years: completed in 4.0 s
        select_years:
        select_years: completed in 0.7 s
        select_lonlat:
        select_lonlat: completed in 2.6 s
    load_data: completed in 1 min 26.1 s
    prepare_XY:
        return_threshold = True        
        make_XY:
            make_X:
            make_X: completed in 0.5 s
            assign_labels:
                A_weights = [3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0]
             

Now onto constructing different architectures

In [41]:
INPUT_DIM = X_va.shape[1:] 
Z_DIM = ut.extract_nested(run_vae_kwargs, 'Z_DIM')
build_encoder_skip_kwargs = ut.extract_nested(run_vae_kwargs, 'build_encoder_skip_kwargs')
build_encoder_skip_kwargs = ut.set_values_recursive(build_encoder_skip_kwargs, {'encoder_conv_filters' : [32,64],
                                                'encoder_conv_kernel_size' : [3,3,64],
                                                'encoder_conv_strides' : [2,1],
                                                'encoder_conv_padding' : ["same","valid"],
                                                'encoder_conv_activation' : ["LeakyRelu","LeakyRelu","LeakyRelu"]
                                                   })
print(build_encoder_skip_kwargs)
_, _, shape_before_flattening, encoder  = tff.build_encoder_skip(input_dim = INPUT_DIM, output_dim = Z_DIM, **build_encoder_skip_kwargs)
encoder.summary()

{'encoder_conv_filters': [32, 64], 'encoder_conv_kernel_size': [3, 3, 64], 'encoder_conv_strides': [2, 1], 'encoder_conv_padding': ['same', 'valid'], 'encoder_conv_activation': ['LeakyRelu', 'LeakyRelu', 'LeakyRelu'], 'encoder_conv_skip': None, 'encoder_use_batch_norm': [False, False, False, False], 'encoder_use_dropout': [0, 0, 0, 0]}
shape_before_flattening =  (10, 22, 64)
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder input (InputLayer)      [(None, 24, 48, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 12, 24, 32)   320         encoder input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu

In [39]:
build_decoder_skip_kwargs = ut.extract_nested(run_vae_kwargs, 'build_decoder_skip_kwargs')
build_decoder_skip_kwargs = ut.set_values_recursive(build_decoder_skip_kwargs, {'decoder_conv_filters' : [32,1],  #3], # Use 3 if working with 3 fields
                                                'decoder_conv_kernel_size' : [3,3],
                                                'decoder_conv_strides' : [1,2],
                                                'decoder_conv_padding' : ["valid","same","same"],
                                                'decoder_conv_activation' : ["LeakyRelu","LeakyRelu","linear"]
                                                   })

ones_dim = np.ones(INPUT_DIM[:-1])
print(f'{shape_before_flattening = }')
filter_mask = np.array([mask,ones_dim,mask], dtype=bool).transpose(1,2,0) 
_, _, decoder = tff.build_decoder_skip(mask=filter_mask, input_dim = Z_DIM, shape_before_flattening = shape_before_flattening, **build_decoder_skip_kwargs)
decoder.summary()

shape_before_flattening = (10, 22, 64)
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 14080)             239360    
_________________________________________________________________
reshape_8 (Reshape)          (None, 10, 22, 64)        0         
_________________________________________________________________
decoder_conv_0 (Conv2DTransp (None, 12, 24, 32)        18464     
_________________________________________________________________
leaky_re_lu_61 (LeakyReLU)   (None, 12, 24, 32)        0         
_________________________________________________________________
decoder_conv_1 (Conv2DTransp (None, 24, 48, 1)         289       
_________________________________________________________________
leaky_re_lu_62 (Leak